# Handgestensteuerung mit MediaPipe-Modul

### Um mit diesem Code den Laptop freihändig zu bedienen (= Bild vergrößern / verkleinern und nächstes Bild anschauen) muss der Code gestartet werden. Die Kamera muss offen sein und es muss bspw. ein Browserfenster geöffnet sein, indem Bilder offen sind. In das Browserfenster muss geklickt werden, damit die Befehle ausgeführt werden können.


In [1]:
import cv2
import mediapipe as mp
import pyautogui

# Initialisiere Mediapipe Hand-Modul
mp_hands = mp.solutions.hands  # Mediapipe Hand-Modul initialisieren
hands = mp_hands.Hands()  # Hand-Erkennung instanziieren
mp_drawing = mp.solutions.drawing_utils  # Zeichen-Utilities von Mediapipe initialisieren

# Öffne die Kamera
cap = cv2.VideoCapture(0)

# Zustandsvariable zur Verfolgung der zuletzt ausgeführten Aktion
last_action = None

def finger_up(landmarks, finger_tip, finger_dip):
    """Prüfe, ob der Finger gestreckt ist"""
    return landmarks[finger_tip].y < landmarks[finger_dip].y #prüfen, ob finger_dip über finger_tip

while cap.isOpened():
    ret, frame = cap.read() #Liest ein Bild von der Kamera ein
    if not ret:
        break #wenn kein Bild gelesen werden kann, Schleife beenden

    # Bild horizontal spiegeln, damit es nicht spiegelverkehrt ist
    frame = cv2.flip(frame, 1)
    

    # Bild von BGR zu RGB konvertieren
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  #Konvertiere das Bild von BGR zu RGB (Mediapipe benötigt RGB)
    image.flags.writeable = False  #Bild als nicht schreibbar markieren, um Performance zu verbessern
    results = hands.process(image)  #Verarbeite das Bild mit Mediapipe Hand-Modul
    image.flags.writeable = True  #Bild wieder als schreibbar markieren
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  #Konvertiere das Bild zurück zu BGR für OpenCV

    if results.multi_hand_landmarks:
        # Wenn Handlandmarken erkannt wurden
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS) # Zeichne Handlandmarken auf das Bild
            
            # Prüfen der Fingerzustände
            landmarks = hand_landmarks.landmark
            index_finger_up = finger_up(landmarks, mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.INDEX_FINGER_DIP) #gestrechter Zeigefinger
            middle_finger_up = finger_up(landmarks, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_DIP) #gestreckter Mittelfinger
            pinky_finger_up = finger_up(landmarks, mp_hands.HandLandmark.PINKY_TIP, mp_hands.HandLandmark.PINKY_DIP) #gestreckter kleiner Finger
            
            if index_finger_up and middle_finger_up and not pinky_finger_up:
                if last_action != 'zoom_in':
                    pyautogui.hotkey('ctrl', '+')  # Vergrößern des Bildes (z.B. in einem Browser oder Bildbetrachter)
                    last_action = 'zoom_in'
            elif pinky_finger_up and not index_finger_up and not middle_finger_up:
                if last_action != 'zoom_out':
                    pyautogui.hotkey('ctrl', '-')  # Verkleinern des Bildes (z.B. in einem Browser oder Bildbetrachter)
                    last_action = 'zoom_out'
            elif index_finger_up and not middle_finger_up:
                if last_action != 'next_slide':
                    pyautogui.press('right')  # Nächste Folie in PowerPoint / nächstes Bild 
                    last_action = 'next_slide'
            else:
                last_action = None  # Reset der Aktion, wenn keine Gesten erkannt werden

    cv2.imshow('Hand Gesture Recognition', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break # Beende die Schleife, wenn die ESC-Taste gedrückt wird

cap.release()  # Kamera freigeben
cv2.destroyAllWindows()  # Alle OpenCV-Fenster schließen


c:\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
